In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.io import ascii
from astropy import units as u

deg = np.pi/180. # radians / degree
yr = 365.25 # days / year

# Astrometric Orbits with Exoplanet

In this notebook we'll walk through the astrometric functionalities of `exoplanet` and how they can be used to fit a variety of systems, including those with parallax and radial velocity measurements. 

To start, we'll start with the simplest example (in terms of number of parameters), which is fitting an orbit of a binary star system that only has astrometric measurements of the separation and orientation of the two stars as a function of time. 

## Data 
First, let's load and examine the data. We'll use the astrometric measurements of HR 466 (HD 10009) as compiled by [Pourbaix 1998](https://ui.adsabs.harvard.edu/#abs/1998A&AS..131..377P/abstract). The speckle observations are originally from [Hartkopf et al. 1996](https://ui.adsabs.harvard.edu/#abs/1996AJ....111..370H/abstract). There are also radial velocities from [Tokovinin 1993](https://ui.adsabs.harvard.edu/#abs/1993AstL...19...73T/abstract), which we'll use later on. We'll follow the conventions from Pourbaix et al., which are a consistent set conforming to the right-hand-rule and the conventions of the visual binary field, where the ascending node is that where the secondary is *receeding* from the observer. 

In [ ]:
# grab the formatted data and do some munging
astro_url = "https://gist.github.com/iancze/262aba2429cb9aee3fd5b5e1a4582d4d/raw/c5fa5bc39fec90d2cc2e736eed479099e3e598e3/astro.txt"
rv1_url = "https://gist.github.com/iancze/262aba2429cb9aee3fd5b5e1a4582d4d/raw/c5fa5bc39fec90d2cc2e736eed479099e3e598e3/rv1.txt"
rv2_url = "https://gist.github.com/iancze/262aba2429cb9aee3fd5b5e1a4582d4d/raw/c5fa5bc39fec90d2cc2e736eed479099e3e598e3/rv2.txt"

astro_data_full = ascii.read(astro_url, format="csv", fill_values=[(".", '0')])

In [ ]:
# convert UT date to JD
astro_dates = Time(astro_data_full["date"].data, format="decimalyear")

# Following the Pourbaix et al. 1998 analysis, we'll limit ourselves to the following highest quality data
# since the raw collection of data outside of these ranges has some ambiguities in swapping 
# the primary and secondary star
ind = (astro_dates.value > 1975.) & (astro_dates.value < 1999.73) \
    & (~astro_data_full["rho"].mask) & (~astro_data_full["PA"].mask)

astro_data = astro_data_full[ind]

astro_yrs = astro_data["date"]
astro_dates.format = 'jd'
astro_jds = astro_dates[ind].value

Many of these measurements do not have associated error measurements. For these, we assume a modest estimate of $5^\circ$ in position angle and $0.05"$ in separation to get a preliminary fit. The scatter in points around the final solution will be a decent guide of what the systematics in the measurement uncertainty actually were.

In [ ]:
astro_data["PA_err"][astro_data["PA_err"].mask == True] = 2.0

In [ ]:
astro_data["rho_err"][astro_data["rho_err"].mask == True] = 0.02

In [ ]:
rho_data = np.ascontiguousarray(astro_data["rho"].data.data, dtype=float) # arcsec
# rho_err = np.array(astro_data["rho_err"].data.data) # arcsec
rho_err = 0.03 * np.ones_like(rho_data)

theta_data = np.ascontiguousarray(astro_data["PA"].data.data * deg, dtype=float) # radians # needs to be in range of -pi to pi
theta_data[theta_data > np.pi] -= 2 * np.pi

# theta_err = np.array(astro_data["PA_err"].data.data * deg) # radians
theta_err = 10.0 * deg * np.ones_like(theta_data)

# and here is the full table
astro_data

In [ ]:
# normally, we assume that rho is measured in degrees east of north

# Make a plot of the astrometric data on the sky

fig, ax = plt.subplots(nrows=1)

xs = rho_data * np.cos(theta_data) # X is north
ys = rho_data * np.sin(theta_data) # Y is east
ax.plot(ys, xs, ".")
ax.set_ylabel(r"$\Delta \delta$ ['']")
ax.set_xlabel(r"$\Delta \alpha \cos \delta$ ['']")
ax.invert_xaxis()
ax.plot(0,0, "k*")
ax.set_aspect("equal", "datalim")

The plot on the sky makes the most sense for interpreting the physical scale of the orbit, but when traditional astrometric measurements are made of a secondary star, they are usually made as the separation and angle from the primary star. This means that the "raw" measurements are the values of $\rho$ (separation) and P.A. (position angle) that we listed in our data table, and that the measurement uncertainties live on these values as nice Gaussians. So, to visualize this space more clearly, we can plot $\rho$ vs. time and P.A. vs. time.

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)
ax[0].plot(astro_yrs, rho_data, ".")
ax[0].errorbar(astro_yrs, rho_data, yerr=rho_err, ls="")
ax[0].set_ylabel(r'$\rho\,$ ["]')

ax[1].plot(astro_yrs, theta_data/deg, ".")
ax[1].errorbar(astro_yrs, theta_data/deg, yerr=theta_err/deg, ls="")
ax[1].set_ylabel(r'P.A. [degrees]');

In [ ]:
# this is the way the orbit is plotted in Hartkopf et al. 1996

# fig, ax = plt.subplots(nrows=1)

# xs = rho_data * np.cos(theta_data * deg) # X is north
# ys = rho_data * np.sin(theta_data * deg) # Y is east
# ax.plot(ys[ind], xs[ind], ".")
# ax.set_ylabel(r"$\Delta \delta$ ['']")
# ax.set_xlabel(r"$\Delta \alpha \cos \delta$ ['']")
# ax.invert_yaxis()
# ax.plot(0,0, "k*")
# ax.set_aspect("equal", "datalim")

In [ ]:
import pymc3 as pm 
import theano.tensor as tt

import exoplanet as xo

import exoplanet.orbits
from exoplanet.distributions import Angle

In [ ]:
astro_jd0 = astro_jds[0]

In [ ]:
astro_jd0

In [ ]:
# Just to get started, let's take a look at the orbit using the best-fit parameters from Pourbaix et al. 1998

# Exoplanet takes omega_1, the star.

# Orbital elements from Pourbaix et al. 1998
# For the relative astrometric fit, we only need the following parameters
a_ang = 0.324 # arcsec
parallax = 1 # arcsec (meaningless choice for now)
a = a_ang * parallax
e = 0.798
i = 96.0 * deg # [rad]
omega = 251.6 * deg - np.pi # omega_2 according to Pourbaix RV equations
Omega = 159.6 * deg 
P = 28.8 * 365.25 # days

T0 = Time(1989.92, format="decimalyear")
T0.format = "jd"
T0 = T0.value # [Julian Date]
Tstart = T0

# n = 2 * np.pi / P
# t0 = (phi + w) / n


# instantiate the orbit
# orbit = xo.orbits.KeplerianOrbit(a=a, t_periastron=T0, period=P, incl=i, ecc=e, omega=omega, Omega=Omega)

# # # make a theano function to get stuff from orbit 
# times = tt.vector("times")

# pos = orbit.get_relative_position(times) # the X, Y, Z measurements
# ang = orbit.get_relative_angles(times) # the rho, theta measurements

# f_pos = theano.function([times], pos)
# f_ang = theano.function([times], ang)

# t = np.linspace(T0 - P, T0 + P, num=200) # days

# # anoms = f_anom(t)
# X, Y, Z = f_pos(t)
# rho, theta = f_ang(t)

In [ ]:
fig, ax = plt.subplots(nrows=1)
ax.plot(Y, X)
ax.invert_xaxis()

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)
ax[0].plot(t, rho)
ax[0].plot(astro_jds, rho_data, ".")
ax[0].errorbar(astro_jds, rho_data, yerr=rho_err, ls="")
ax[0].set_ylabel(r'$\rho\,$ ["]')

theta_wrap = theta 
theta_wrap[theta_wrap < 0] += 2 * np.pi

ax[1].plot(t, theta/deg)
ax[1].plot(astro_jds, theta_data/deg, ".")
ax[1].errorbar(astro_jds, theta_data/deg, yerr=theta_err/deg, ls="")
ax[1].set_ylabel(r'P.A. [degrees]');

In [ ]:
fig, ax = plt.subplots(nrows=1)
ax.plot(Y, X, "o")
ax.plot(0, 0, "k*")
ax.invert_xaxis()
ax.set_xlabel(r"$\Delta \alpha \cos \delta$ [arcsec]")
ax.set_ylabel(r"$\Delta \delta$ [arcsec]");
ax.set_aspect("equal", "datalim")

In [ ]:
theta_data

Orbit fitting tip from Mike Fitzgerald to use $\Omega+\omega$ and $\Omega-\omega$ for fitting, since you can get unambiguous constraints on those parameters. 

Without radial velocity information, there is a $180^\circ$ degeneracy in the position angle of the ascending node, $\Omega_2$.

In [ ]:
# Sample the orbit 

with pm.Model() as model:

    # A setup for a basic astrometric binary orbit
    parallax = 1 # arcsec 
    a_ang = pm.Uniform("a_ang", 0.1, 1.0, testval=0.324) # arcsec 
    a = pm.Deterministic("a", a_ang * parallax)
        
    logP = pm.Uniform("logP", lower=np.log(10 * 365.25), upper=np.log(80 * 365.25), testval=np.log(28.8 * 365.25))
    P = pm.Deterministic("P", tt.exp(logP))
    
    omega = Angle("omega", testval=251.6 * deg - np.pi) # - pi to pi
    Omega = Angle("Omega", testval=159.6 * deg) # - pi to pi
    
    # fit for (Ω + ω) and (Ω - ω)
#     Omega_plus_omega = Angle("OmegaPlusomega")
#     Omega_minus_omega = Angle("OmegaMinusomega")
    
#     omega = pm.Deterministic("omega", 0.5 *(Omega_plus_omega - Omega_minus_omega))
#     Omega = pm.Deterministic("Omega", 0.5 * (Omega_plus_omega + Omega_minus_omega))
    
    # orbital phase 
    phi = Angle("phi", testval=1.605) # - pi to pi
    
    n = 2 * np.pi * tt.exp(-logP) # radians / day
    t_periastron = pm.Deterministic("tperi", (phi + omega) / n) # a reference time for periastron passage
#     t_periastron = pm.Normal("tperi", mu=Tstart - astro_jd0, sd=10)
    
    incl = pm.Uniform("incl", lower=0, upper=np.pi, testval=96.0 * deg) # radians, 0 to 180 degrees
    e = pm.Uniform("e", lower=0.0, upper=1.0, testval=0.798)
           
    orbit = xo.orbits.KeplerianOrbit(a=a, t_periastron=t_periastron, period=P, 
                                   incl=incl, ecc=e, omega=omega, Omega=Omega)
    
    rho_model, theta_model = orbit.get_relative_angles(astro_jds - astro_jd0) # the rho, theta model values
    
    # to account for broadcasting to multiple planets
    rho_model = tt.sum(rho_model, axis=-1)
    theta_model = tt.sum(theta_model, axis=-1)
    
    pm.Normal("obs_rho", mu=rho_model, observed=rho_data, sd=rho_err)
    pm.Normal("obs_theta", mu=theta_model, observed=theta_data, sd=theta_err)

In [ ]:
model.test_point

In [ ]:
with model:
    map_sol = xo.optimize(vars=[phi])
#     map_sol = xo.optimize(map_sol, vars=[omega, Omega])

In [ ]:
map_sol["phi"]

In [ ]:
plt.plot(astro_jds, rho_data, "o")
plt.errorbar(astro_jds, rho_data, yerr=rho_err, ls="")
with model:
#     plt.plot(astro_jds, xo.eval_in_model(rho_model))
    plt.plot(astro_jds, xo.eval_in_model(rho_model, map_sol))

In [ ]:
with model:

    plt.plot(astro_jds, (rho_data - xo.eval_in_model(rho_model)) / rho_err, "o")
    plt.plot(astro_jds, (rho_data - xo.eval_in_model(rho_model, map_sol)) / rho_err, "o")

In [ ]:
plt.plot(astro_jds, theta_data, "o")
plt.errorbar(astro_jds, theta_data, yerr=theta_err, ls="")
with model:
    plt.plot(astro_jds, xo.eval_in_model(theta_model))
    plt.plot(astro_jds, xo.eval_in_model(theta_model, map_sol))

In [ ]:
print(map_sol)

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)
# ax[0].plot(t, rho)
ax[0].plot(astro_jds, rho_data, ".")
ax[0].errorbar(astro_jds, rho_data, yerr=rho_err, ls="")
ax[0].set_ylabel(r'$\rho\,$ ["]')

# ax[1].plot(t, theta/deg)
ax[1].plot(astro_jds, theta_data/deg, ".")
ax[1].errorbar(astro_jds, theta_data/deg, yerr=theta_err/deg, ls="")
ax[1].set_ylabel(r'P.A. [degrees]');


with model:    
    ax[0].plot(astro_jds, xo.eval_in_model(rho_model, map_sol), "k", alpha=0.5)
    ax[1].plot(astro_jds, xo.eval_in_model(theta_model, map_sol)/deg, "k", alpha=0.5)

In [ ]:
# with model:
# #     step = pm.Metropolis()
# #     trace = pm.sample(draws=5000, tune=1000, step=step, chains=1)
    
#     trace = pm.sample(draws=1000, tune=1000, chains=2)
    
sampler = xo.PyMC3Sampler(start=200, window=100, finish=300)
with model:
    burnin = sampler.tune(tune=4000, start=model.test_point,
                          step_kwargs=dict(target_accept=0.95))
    trace = sampler.sample(draws=4000)

In [ ]:
pm.traceplot(trace)